In [1]:
# import library
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pulp
import random
import math
import pdb
import time
import os

In [2]:
def read_link_tobe_served():
    global RoutePlan,newRoutePlan
    file=open('SnowplowRoutes.dat','r')
    RoutePlan=[]
    newRoutePlan=[]
    for line in file:
        line_list_temp=line.split()
        if len(line_list_temp)>2:
            RoutePlan.append([int(j) for j in line_list_temp])
    for route in RoutePlan:
        route_length=0
        i=0
        j=0
        for link in route:
            if route_length<=35*3:
                if link<=num_links:
                    route_length=route_length+link_detail[link,4]/5280
                if j==len(route)-1:
                    newRoutePlan.append(route[i:j])
                    print(route[i:j],1)
            else: 
                if route[i:j]==[]:
                    pdb.set_trace()
                newRoutePlan.append(route[i:j])
                print(route[i:j],2)
                route_length=0
                i=j+1
            j=j+1
#     LtbSFile=open('../newRoutePlan.dat','w')
    LtbSFile=open('newRoutePlan.dat','w')
    print(LtbSFile)
    print(len(newRoutePlan),file=LtbSFile)
    for route in newRoutePlan:
        print (" ".join(str(x) for x in route),file=LtbSFile)
    return
def convert_route_tobe_served():
    global LinkTobeServedList,count
    simulation_length=0.1
    dynasmart_num_simulation_interval=int(1440.0/simulation_length)
    get_start_time()
    num_links=4805
    LinkTobeServedList=[]
    for i in range(math.floor(dynasmart_num_simulation_interval)):
        LinkTobeServedList.append([])
    count=0
    for route in RoutePlan:
        service_finish_time_temp=service_start_time
        for link in route: 
            if link<=num_links:
                service_finish_time_temp=service_finish_time_temp+link_detail[link,4]/5280/0.5
                if service_finish_time_temp>service_start_time+180:
                    service_finish_time_temp=service_start_time
                    count=count+1
                LinkTobeServedList[math.floor(service_finish_time_temp/simulation_length)].extend([link])
#     LtbSFile=open('../LinkToBeServed.dat','w')
    LtbSFile=open('LinkToBeServed.dat','w')
    print(dynasmart_num_simulation_interval,len(max(LinkTobeServedList)),file=LtbSFile)
    for i in range(math.floor(dynasmart_num_simulation_interval)):
        LinkTobeServedList[i].extend([-1]*(max(len(p) for p in LinkTobeServedList)-len(LinkTobeServedList[i])))
        print (" ".join(str(x) for x in LinkTobeServedList[i]),file=LtbSFile)
    
    LtbSFile.close() 
    return

def get_start_time():
    global service_start_time
    if dynasmart_method==0: 
        snow_depth=0
        i=0
        while snow_depth<=0.2:
            snow_depth=snow_detail[i][0]/60*5+snow_depth
            i=i+1
        service_start_time=snow_detail[i][1]
        print('Dynasmart P service start time',service_start_time)
    elif dynasmart_method==1:
        #read the service start time
        file=open('CurrentDynasmartTime.dat')
        for line in file: 
            service_start_time=float(line)
        print('Dynasmart P service start time',service_start_time)

In [3]:
def read_network():
    '''
        input: network.dat
        output:
            num_zones: number of zones within the network
            num_nodes: number of nodes in the network
            num_links: number of links in the network
            node_detail: node id and zone which the node belong to
            node_id: match the node id to 1~num_nodes to save memory
            link_detail:The information associated with a specific link
                link_detail[0] upstream node
                link_detail[1] downstream node
                link_detail[2] Num of left turn bay
                link_detail[3] Num of right turn bay
                link_detail[4] length of link
                link_detail[5] Num of lanes
                link_detail[6] Traffic flow mode
                link_detail[7] posted speed limit adjustment margin
                link_detail[8] posted speed limit
                link_detail[9] maximum service flow rate for the first link
            link_id: a matrix to find a specific link based on origin node nd destination node 
   '''
    global num_zones,num_nodes,num_links,node_detail,node_id,link_detail,link_id
    file=open('network.dat','r')
    i=0
    network_basic=list()
    node_list=list()
    node_id={}
    link_detail_list=list()
    for line in file:
        line_list_temp=line.split()
        # Read the first line of network.dat
        if i==0:
            num_zones=int(line_list_temp[0])
            num_nodes=int(line_list_temp[1])
            num_links=int(line_list_temp[2]) #+5 # +5 because 5 nodes can not be visited from anyother link, therefore we manually creat few link to connect it to the network
            num_shortest_path=int(line_list_temp[3])
            zone_flag=int(line_list_temp[4])
            link_id=np.zeros((num_nodes,num_nodes))
        #Read the node information from network.dat
        elif len(line_list_temp)<3:
            node_list.append([int(line_list_temp[0]),int(line_list_temp[1])])
            node_id.update({int(line_list_temp[0]):i-1})
        #Read the link information from network.dat
        else:
            line_list_temp=[float(j) for j in line_list_temp]
            link_detail_list.append(line_list_temp)
            link_id[node_id[line_list_temp[0]],node_id[line_list_temp[1]]]=i-num_nodes-1
        i=i+1
    ##############################
    #serveral nodes can not be visited from anyother link, therefore we manually creat few link to connect it to the network
#     link_detail_list.append([15293,15315,0,0,2428,1,9,0,30,2000,1800,5,0])
#     link_detail_list.append([12816,12828,0,0,2428,1,9,0,30,2000,1800,5,0])
#     link_detail_list.append([13531,13513,0,0,2428,1,9,0,30,2000,1800,5,0])
#     link_detail_list.append([13643,13642,0,0,2428,1,9,0,30,2000,1800,5,0])
#     link_detail_list.append([16115,16135,0,0,2428,1,9,0,30,2000,1800,5,0])
#     link_id[node_id[15293],node_id[15315]]=num_links-5
#     link_id[node_id[12816],node_id[12828]]=num_links-4
#     link_id[node_id[13531],node_id[13513]]=num_links-3
#     link_id[node_id[13643],node_id[13642]]=num_links-2
#     link_id[node_id[16115],node_id[16135]]=num_links-1

    ##############################
    node_detail=np.matrix(node_list)
    link_detail=np.matrix(link_detail_list)
   
    return  #num_zones,num_nodes,num_links,node_detail,node_id,link_detail,link_id
def read_flow():
    '''
        input: outflow.dat
        output: 
            link_volume - matrix
    '''
    global link_volume
    volume_time=[]
    file=open('OutFlow.dat')
    i=1
    link_volume=[]
    link_volume_one_time_interval=[]
    for line in file:
        line_list=line.split()
        if i>6 : # skip the first 6 lines
            if len(line_list)==1: #find the line indicating time interval
                volume_time.append(float(line_list[0]))
#                 link_volume_one_time_interval.extend([0.01,0.01,0.01,0.01,0.01])  #serveral nodes can not be visited from anyother link, therefore we manually creat few link to connect it to the network
                #print(line_list[0],len(link_volume_one_time_interval))
                link_volume.append(link_volume_one_time_interval)
                link_volume_one_time_interval=[]
            else:
                line_list=[float(j) for j in line_list]
                link_volume_one_time_interval.extend(line_list)
        i=i+1
#     link_volume_one_time_interval.extend([35,35,35,35,35])  #serveral nodes can not be visited from anyother link, therefore we manually creat few link to connect it to the network
    link_volume.append(link_volume_one_time_interval)
    link_volume.pop(0)
    link_volume=np.array(link_volume)
    return link_volume
def read_speed():
    '''
        input:OutLinkSpeedAll.dat
        output: 
            link_speed -matrix
    '''
    global link_speed
    speed_time=[]
    file=open('OutLinkSpeedAll.dat')
    i=1
    link_speed=[]
    link_speed_one_time_interval=[]
    for line in file:
        line_list=line.split()
        if i>6 : # skip the first 6 lines
            if len(line_list)==1: #find the line indicating time interval
                speed_time.append(float(line_list[0]))
#                 link_speed_one_time_interval.extend([0,0,0,0,0])  #serveral nodes can not be visited from anyother link, therefore we manually creat few link to connect it to the network
                link_speed.append(link_speed_one_time_interval)
                link_speed_one_time_interval=[]
            else:
                line_list=[float(j) for j in line_list]
                link_speed_one_time_interval.extend(line_list)
        i=i+1
#     link_speed_one_time_interval.extend([0,0,0,0,0])  #serveral nodes can not be visited from anyother link, therefore we manually creat few link to connect it to the network
    link_speed.append(link_speed_one_time_interval)
    link_speed.pop(0)
    link_speed=np.array(link_speed)
    return link_speed

def read_xy():
    '''
        input: xy.dat
        output: nodexy: a dictionary storing the latitude and longtitude of 
            each node
    '''
    global nodexy
    nodexy={}
    file=open('xy.dat')
    for line in file:
        line_list_temp=line.split()
        line_list_temp=[float(j) for j in line_list_temp]
        nodexy.update({line_list_temp[0]:[line_list_temp[1]/1000000.0*51.33,-line_list_temp[2]/1000000.0*68]})
    return
def read_snow():
    '''
        ipnut: weather.dat
        output: snow_detail. 
                    snow_detail[i,0] snow intensity at time interval i
                    snow_detail[i,1] start time of time interval i
                    snow_detail[i,2] end time of time interval i
    '''
    global snow_detail,snow_interval
    file=open('weather.dat')
    snow_detail=[]
    i=0
    for line in file: 
        if i==0:
            num_interval=float(line.split()[0])
            #print(num_interval,i,line)
            snow_interval=planning_horizon/num_interval
        elif i!= num_interval+1:
            line_list_temp= line.split()
            snow_detail.append([float(line_list_temp[2]),float(line_list_temp[3]),float(line_list_temp[4])])
        i=i+1
    return
def read_snowaccum(): 
    global sa_factors
    sa_factors=[]
    file=open('SnowAccuFactor.dat')
    for line in file: 
        if len(line)>4:
            line_list_temp=[j for j in line.split()]
            sa_factors.append([float(j) for j in line_list_temp])
    return
# def read_scenario()
#     '''
#         Input: Scenario.dat
#         output:
#     '''
def read_snow():
    '''
        ipnut: weather.dat
        output: snow_detail. 
                    snow_detail[i,0] snow intensity at time interval i
                    snow_detail[i,1] start time of time interval i
                    snow_detail[i,2] end time of time interval i
    '''
    global snow_detail,snow_interval
    file=open('weather.dat')
    snow_detail=[]
    i=0
    for line in file: 
        if i==0:
            num_interval=float(line.split()[0])
            #print(num_interval,i,line)
            snow_interval=planning_horizon/num_interval
        elif i!= num_interval+1:
            line_list_temp= line.split()
            snow_detail.append([float(line_list_temp[2]),float(line_list_temp[3]),float(line_list_temp[4])])
        i=i+1
    return

In [4]:
os.getcwd()

'D:\\alex\\AMS Project\\SP routing\\Full network - oc6 new'

In [5]:
read_network()

In [6]:
read_link_tobe_served()

[3807, 3775, 3747, 3732, 3669, 3899, 3946, 3943, 3940, 3937, 3935, 4282, 3932, 3893, 3845, 3805, 3771, 3684, 3721, 4806, 3770, 3746, 3771, 3684, 3721, 4806, 3770, 3746, 3771, 3684, 3721, 4806, 3770, 3746, 3771, 3684, 3721, 4806, 3770, 3746, 3771, 3684, 3721, 4806, 3770, 3746, 3773, 3806, 3847, 3895, 3935, 4282, 3932, 3893, 3846, 3848, 3807, 3775, 3747, 3732, 3669, 3899, 3946, 3943, 3940, 3937, 3935, 4282, 3932, 3893, 3846, 3848, 3807, 3775, 3747, 3732, 3669, 3900, 4121, 4164, 4278, 4341, 4405, 4403, 4497, 4495, 4593, 4664, 4712, 4728, 4703, 4722, 4739, 4740, 4715, 4731, 4728, 4703, 4722, 4739, 4740, 4715, 4731, 4728, 4703, 4722, 4739, 4740, 4715, 4731, 4728, 4703, 4722, 4739, 4740, 4715, 4731, 4729, 4713, 4739, 4740, 4715, 4731, 4729, 4713, 4739, 4741, 4733, 4737, 4766, 4768, 4769, 4750, 4710, 4517, 4475, 4346, 4396, 4505, 4700, 4755, 4752, 4751, 4718, 4747, 4737, 4766, 4768, 4769, 4750, 4710, 4517, 4475, 4346, 4396, 4505, 4700, 4755, 4752, 4751, 4718, 4747, 4737, 4766, 4768, 4769, 475

In [7]:
totallength=0
for route in RoutePlan:
    for link in route: 
        if link<=4805:
            totallength=totallength+link_detail[link,4]

In [8]:
dynasmart_method=0
planning_horizon=1440.0
read_snow()
convert_route_tobe_served()

Dynasmart P service start time 400.0


In [9]:
totallength/5280

3822.4479166666665

In [10]:
lanlength=[link_detail[i,4]*link_detail[i,5]/5280 for i in range(4805)]
totallanlength=sum(lanlength)

In [11]:
totallanlength

2235.6236742424244